In [4]:
from pathlib import Path
import pandas as pd
from prefect import flow, task
from prefect_gcp.cloud_storage import GcsBucket
from prefect_gcp import GcpCredentials
from datetime import date

def extract_from_gcs():
    """Download youtube data from GCS"""
    today = date.today()
    today = str(today).replace("-", "")
    local =f"/home/testuser/dezoomcamp-project-youtube/4_upload_data_from_gcs_to_bq/{today}.parquet"
    gcs_path = f'youtube_data/{today}.parquet'
    gcs_block = GcsBucket.load("zoom-gcs")
    gcs_block.get_directory(from_path=gcs_path, local_path=local)
    print(Path(f"{gcs_path}"))
    return Path(f"{local}")

def transform(path: Path) -> pd.DataFrame:
    """Data cleaning example"""
    df = pd.read_parquet(path)
    print (len(df))
    return df

def write_bq(df: pd.DataFrame):
    """Write DataFrame to BiqQuery"""
    gcp_credentials_block = GcpCredentials.load("zoom-gcp-creds")
    df.to_gbq(
    destination_table= "youtube_data_all.youtube_trends",
    project_id="dtc-de-youtube",
    credentials=gcp_credentials_block.get_credentials_from_service_account(),
    chunksize=500_000,
    if_exists="append",
    )
    

def etl_gcs_to_bq():
    """Main ETL flow to load data into Big Query"""
    path = extract_from_gcs()
    df = transform(path)
    write_bq(df)

In [1]:
#!pip install pandas-gbq==0.18.1

In [7]:
path = extract_from_gcs()

AttributeError: 'coroutine' object has no attribute 'get_directory'

In [ ]:
df = transform(path)

In [ ]:
write_bq(df)